# Quart + Jinja2 + Three.js Chart Demo

This notebook demonstrates how to:
- Use **Pandas** to create sample data
- Build an async web server with **Quart**
- Render HTML templates with **Jinja2**
- Visualize data with **Three.js** 3D charts
- Expose the server via **ngrok** for Colab access

**Goal**: Show a simple, clean pattern for building interactive 3D visualizations.

## 1. Install Dependencies

Install Quart (async web framework) and pyngrok (for tunneling).

In [ ]:
!pip install -q quart pyngrok pandas numpy requests


## 2. Create Sample Data

Generate simple sales data using Pandas.

In [ ]:
import pandas as pd
import numpy as np

# Create sample quarterly sales data for different products
data = {
    'Product': ['Product A', 'Product B', 'Product C', 'Product D'],
    'Q1': [45, 60, 38, 52],
    'Q2': [58, 72, 45, 61],
    'Q3': [62, 68, 51, 58],
    'Q4': [71, 85, 59, 67]
}

df = pd.DataFrame(data)
print("Sample Data:")
print(df)

# Convert to format suitable for JavaScript
chart_data = df.to_dict('records')
print("\nData for chart:", chart_data)

## 3. Set Up Quart Application

Create a Quart app with a single route that renders our chart.

In [ ]:
from quart import Quart, render_template_string
import json
import requests

app = Quart(__name__)

# Load HTML template from repository
TEMPLATE_URL = 'https://raw.githubusercontent.com/andrewcgaitskell/colab-threejs-charting/main/templates/chart_template.html'
response = requests.get(TEMPLATE_URL, timeout=10)
HTML_TEMPLATE = response.text

@app.route('/')
async def index():
    """Main route that renders the 3D chart with data"""
    return await render_template_string(HTML_TEMPLATE, data=chart_data)

print("✓ Quart app configured")


## 4. Set Up Ngrok Tunnel

Create a public URL to access the Quart server from Colab.

In [ ]:
from pyngrok import ngrok
import nest_asyncio

# Allow nested async loops (required for Colab)
nest_asyncio.apply()

# Start ngrok tunnel
port = 5000
public_url = ngrok.connect(port)
print(f"🌐 Public URL: {public_url}")
print(f"\nClick the link above to view your 3D chart!")

## 5. Run the Server

Start the Quart server and display the visualization.

**Note**: This cell will run continuously. Click the link above to view the chart.

In [ ]:
import asyncio
from IPython.display import HTML, display

# Display clickable link
display(HTML(f'<h2><a href="{public_url}" target="_blank">🚀 Open 3D Chart Visualization</a></h2>'))

# Run the Quart app
print("Server running... Press Ctrl+C to stop")
await app.run_task(host='0.0.0.0', port=port, debug=False)

## 6. Cleanup (Optional)

Stop the ngrok tunnel when done.

In [ ]:
# Stop ngrok tunnel
ngrok.kill()
print("Ngrok tunnel closed")

---

## Summary

This notebook demonstrates:

1. **Data Pipeline**: Pandas → Python dict → JSON → JavaScript
2. **Web Framework**: Quart (async) with Jinja2 templating
3. **3D Visualization**: Three.js for interactive charts
4. **Deployment**: Ngrok for public access from Colab

### Key Features:
- ✅ Simple, minimal code
- ✅ Interactive 3D controls (drag to rotate, scroll to zoom)
- ✅ Clean data flow from Python to JavaScript
- ✅ Professional presentation-ready output

### Customization Ideas:
- Change the data in Step 2
- Modify colors in the Three.js code
- Add more chart types (scatter, line, etc.)
- Enhance with additional Three.js features (labels, animations, etc.)
